In [ ]:
elastic_rods_dir = '../../../elastic_rods/python/'
weaving_dir = '../../'
import os
import os.path as osp
import sys; sys.path.append(elastic_rods_dir); sys.path.append(weaving_dir)
import numpy as np, elastic_rods, linkage_vis
import numpy.linalg as la
from bending_validation import suppress_stdout as so
import matplotlib.pyplot as plt
from elastic_rods import EnergyType, InterleavingType

# weaving
import analysis_helper, ribbon_linkage_helper, mesh_vis, linkage_utils, compute_curve_from_curvature, pipeline_helper, importlib
importlib.reload(analysis_helper)
importlib.reload(ribbon_linkage_helper)
importlib.reload(mesh_vis)
importlib.reload(linkage_utils)
importlib.reload(compute_curve_from_curvature)
importlib.reload(pipeline_helper)
from analysis_helper import (compare_turning_angle,
                            is_on_sphere, 
                            get_distance_to_center_scalar_field, 
                            plot_curvatures, 
                            get_curvature_scalar_field,
                            construct_elastic_rod_loop_from_rod_segments, 
                            concatenate_rod_properties_from_rod_segments, 
                            compute_min_distance_rigid_transformation)
from ribbon_linkage_helper import (update_rest_curvature, 
                                   set_ribbon_linkage,
                                   export_linkage_geometry_to_obj,
                                   write_linkage_ribbon_output_florin)
from compute_curve_from_curvature import (match_geo_curvature_and_edge_len, get_all_curve_pattern)
from linkage_utils import order_segments_by_ribbons, get_turning_angle_and_length_from_ordered_rods

from pipeline_helper import (initialize_linkage, get_normal_deviation, set_joint_vector_field, stage_1_optimization, initialize_stage_2_optimizer, stage_2_optimization, InputOrganizer, write_all_output, set_surface_view_options, get_structure_analysis_view, get_double_side_view, show_selected_joints)
import vis.fields
import matplotlib.cm as cm
import time

In [ ]:
rod_length = 0.09982061776030171
width = rod_length / 15 * 5
thickness = width / 5 * 0.35
print(width, thickness)
io = InputOrganizer('owl_1', thickness, width, weaving_dir)
#  [ ear 214, ear 255, nose 39]

In [ ]:
import py_newton_optimizer
OPTS = py_newton_optimizer.NewtonOptimizerOptions()
OPTS.gradTol = 1e-6
OPTS.verbose = 1;
OPTS.beta = 1e-8
OPTS.niter = 200
OPTS.verboseNonPosDef = False
rw = 1
sw = 10
drw = 0.1
dsw = 0.1

In [ ]:
# import pickle 
# import gzip
# curved_linkage = pickle.load(gzip.open('{}_stage_2.pkl.gz'.format(io.MODEL_NAME), 'r'))

In [ ]:
with so(): curved_linkage = initialize_linkage(surface_path = io.SURFACE_PATH, useCenterline = True, model_path = io.MODEL_PATH, cross_section = io.RIBBON_CS, subdivision_res = io.SUBDIVISION_RESOLUTION, use_constant_width=True, interleaving_type=elastic_rods.InterleavingType.weaving)
curved_linkage.set_design_parameter_config(use_restLen = True, use_restKappa = True)
curved_save_tgt_joint_pos = curved_linkage.jointPositions();
curved_linkage_view = linkage_vis.LinkageViewerWithSurface(curved_linkage, io.SURFACE_PATH)
set_surface_view_options(curved_linkage_view)
curved_linkage_view.show()

In [ ]:
curved_linkage.setExtendedDoFsPSRL(np.load('owl_1_dof.npy'))

In [ ]:
bottomColor =[180/255., 158/255., 119/255.]
topColor =[79/255., 158/255., 246/255.]
heights = curved_linkage.visualizationGeometryHeightColors()
colors = np.take(np.array([bottomColor, topColor]), heights < heights.mean(), axis=0)

In [ ]:
def eqm_callback(prob, i):
    curved_linkage_view.update(scalarField = colors)

In [ ]:
elastic_rods.compute_equilibrium(curved_linkage, callback = eqm_callback, options = OPTS)

In [ ]:
def visualize_crossing(index, linkage, view):
    vf = show_selected_joints(linkage, [index], flip = True)
    view.update(vectorField = vf, scalarField = colors)

In [ ]:
ribbons = order_segments_by_ribbons(curved_linkage)
_, _, _, _, all_joint_index, _ = get_turning_angle_and_length_from_ordered_rods(ribbons, curved_linkage, rest = True)
all_joint_index_list = [j_list + [j_list[0]] for j_list in all_joint_index]

def get_ribbon_crossing_list(index, linkage, view):
    selected_list = []
    selected_ribbon = []
    for ribbon_index, index_list in enumerate(all_joint_index_list):
        if index in set(index_list):
            selected_ribbon.append(ribbon_index)
            selected_list.append(index_list)
    print("The crossing {} belongs to ribbon {}".format(index, ', '.join([str(x) for x in selected_ribbon])))
    for i in range(len(selected_list)):
        print('Ribbon {}: {}'.format(selected_ribbon[i], selected_list[i]))

In [ ]:
def select_crossing(crossing_index):
    visualize_crossing(crossing_index, curved_linkage, curved_linkage_view)
    get_ribbon_crossing_list(crossing_index, curved_linkage, curved_linkage_view)
select_crossing(85)

In [ ]:
def write_crossing_ribbon_info(linkage, filename, scale):
    ''' return the list of two ribbon indices per crossing and the list of crossing lists per ribbon
    '''
    ribbons = order_segments_by_ribbons(linkage)
    _, _, _, _, all_joint_index, _ = get_turning_angle_and_length_from_ordered_rods(ribbons, linkage, rest = True)
    all_joint_index_list = [j_list + [j_list[0]] for j_list in all_joint_index]

    def get_ribbon_crossing_list(index):
        selected_list = []
        selected_ribbon = []
        for ribbon_index, index_list in enumerate(all_joint_index_list):
            if index in set(index_list):
                selected_ribbon.append(ribbon_index)
                selected_list.append(index_list)
        # print("The crossing {} belongs to ribbon {}".format(index, ', '.join([str(x) for x in selected_ribbon])))
        # for i in range(len(selected_list)):
        #     print('Ribbon {}: {}'.format(selected_ribbon[i], selected_list[i]))
        return selected_ribbon
    pairs_of_ribbons_per_crossing = [get_ribbon_crossing_list(i) for i in range(linkage.numJoints())]

    with open(filename, 'w') as f:
        f.write('# crossing count {}, ribbon count {}'.format(linkage.numJoints(), len(ribbons)))
        for vx in linkage.jointPositions().reshape((linkage.numJoints(), 3)):
            vx *= scale
            f.write('v  {} {} {}\n'.format(vx[0], vx[1], vx[2]))
        for c_index, pair in enumerate(pairs_of_ribbons_per_crossing):
            if len(pair) == 1:
                f.write('crossing {}: {}\n'.format(c_index, pair[0]))
            else:
                if len(pair) != 2:
                    print("Something is wrong")
                f.write('crossing {}: {} {}\n'.format(c_index, pair[0], pair[1]))
        for r_index, index_list in enumerate(all_joint_index_list):
            f.write('ribbon {}: {}\n'.format(r_index, " ".join([str(x) for x in index_list])))

    return pairs_of_ribbons_per_crossing, all_joint_index_list

In [ ]:
two_ribbon_per_crossing, crossing_list_per_ribbon = write_crossing_ribbon_info(curved_linkage, 'cat_webapp_output.txt', 100)